In [24]:
import os
import pandas as pd
import numpy as np
import xarray as xr
from scipy import spatial
import random

In [45]:
import dask.distributed as dd

client = dd.Client("tcp://129.82.20.48:8786")

In [46]:
client

<Client: 'tcp://129.82.20.48:8786' processes=30 threads=60, memory=150.00 GiB>

In [43]:
client.shutdown()

In [27]:
runs = [
    r
    for r in sorted(os.listdir(f"/moonbow/gleung/incusres-analysis/"))
    if os.path.exists(
        f"/moonbow/gleung/incusres-analysis/{r}/tobac-filtered-fts-rev-g3.pkl"
    )
    and "-R-" in r
]

outx = pd.read_pickle("/moonbow/gleung/incusres/nx.pkl")
outy = pd.read_pickle("/moonbow/gleung/incusres/ny.pkl")

dxy = 100

In [31]:
runs = [
    "ARG1.2-R-V0",
    "AUS1.1-R-V0",
    "BRA1.1-R-V0",
    "BRA1.2-R-V0",
    "DRC1.1-R-V0",
    "PHI1.1-R-V0",
    "PHI2.1-R-V0",
    "RSA1.1-R-V0",
    "USA1.1-R-V0",
    "WPO1.1-R-V0",
] 

In [47]:
grids = ["g2"]
dxys = [400]

for grid, dxy in zip(grids, dxys):
    for run in runs[1:-2][::-1]:
        if not os.path.isdir(f"/moonbow/gleung/incusres/{run}/{grid}/fullgrid/"):
            os.mkdir(f"/moonbow/gleung/incusres/{run}/{grid}/fullgrid/")

        Features = pd.read_parquet(
            f"//monsoon/MODEL/LES_MODEL_DATA/Tracking/V0/{run}/{grid}/w_features.pq"
        )

        Features["time"] = pd.to_datetime(Features["timestr"])
        Features["x"] = Features.hdim_2 * dxy
        Features["y"] = Features.hdim_1 * dxy

        nn_dists = []
        times = sorted(Features.time.unique())
        ts = times  # random.sample(times, 30)

        for t in ts:
            if not os.path.exists(f"/moonbow/gleung/incusres/{run}/{grid}/fullgrid/{t.strftime('%Y%m%d-%H%M%S')}.csv"):
                ft = Features[Features.time == t]

                centroids = ft[["y", "x", "ztn"]].values

                tree = spatial.KDTree(centroids)

                out = client.map(tree.query, centroids, k=2)

                out = client.gather(out)
                out = np.array(out)[:, 0, 1]

                np.savetxt(
                    f"/moonbow/gleung/incusres/{run}/{grid}/fullgrid/{t.strftime('%Y%m%d-%H%M%S')}.csv",
                    out,
                )

2024-08-26 14:14:09,700 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
